In [1]:
from dataset.build_dataset import build_dataset
import mindspore.dataset as ds
import os
import numpy as np
from mindspore.mindrecord import FileWriter
import json

[WARNING] ME(28821:140430176667456,MainProcess):2021-10-25-16:37:29.103.813 [mindspore/run_check/_check_version.py:181] Cuda ['10.1', '11.1'] version(need by mindspore-gpu) is not found, please confirm that the path of cuda is set to the env LD_LIBRARY_PATH, please refer to the installation guidelines: https://www.mindspore.cn/install
[WARNING] ME(28821:140430176667456,MainProcess):2021-10-25-16:37:29.111.449 [mindspore/run_check/_check_version.py:181] Cuda ['10.1', '11.1'] version(need by mindspore-gpu) is not found, please confirm that the path of cuda is set to the env LD_LIBRARY_PATH, please refer to the installation guidelines: https://www.mindspore.cn/install


In [2]:
FEATURES_FILE = "./data/train_data.npy"

features = np.load(FEATURES_FILE)
list_dict = []
for item in features:
    dict_temp = json.loads(item)
    list_dict.append(dict_temp)

In [3]:
len(list_dict[0]['word_ids'])

180

In [4]:
list_dict[0]['word_ids']

[0,
 598,
 2661,
 222,
 5,
 9880,
 2708,
 2346,
 2082,
 11,
 504,
 4432,
 11,
 226,
 2126,
 10067,
 1470,
 116,
 2,
 2,
 29474,
 28108,
 6,
 5,
 334,
 34,
 10,
 4019,
 2048,
 4,
 497,
 1517,
 5,
 4326,
 6919,
 18,
 1637,
 31346,
 16,
 10,
 9030,
 9577,
 9,
 5,
 9880,
 2708,
 4,
 29261,
 11,
 760,
 9,
 5,
 4326,
 6919,
 8,
 2114,
 24,
 6,
 16,
 10,
 7621,
 9577,
 9,
 4845,
 19,
 3701,
 62,
 33161,
 19,
 5,
 7875,
 22,
 39043,
 1459,
 1614,
 1464,
 13292,
 4977,
 845,
 4130,
 7,
 5,
 4326,
 6919,
 16,
 5,
 26429,
 2426,
 9,
 5,
 25095,
 6924,
 4,
 29261,
 639,
 5,
 32394,
 2426,
 16,
 5,
 7461,
 26187,
 6,
 10,
 19035,
 317,
 9,
 9621,
 8,
 12456,
 4,
 85,
 16,
 10,
 24633,
 9,
 5,
 11491,
 26187,
 23,
 226,
 2126,
 10067,
 6,
 1470,
 147,
 5,
 9880,
 2708,
 2851,
 13735,
 352,
 1382,
 7,
 6130,
 6552,
 625,
 3398,
 208,
 22895,
 853,
 1827,
 11,
 504,
 4432,
 4,
 497,
 5,
 253,
 9,
 5,
 1049,
 1305,
 36,
 463,
 11,
 10,
 2228,
 516,
 14,
 15230,
 149,
 155,
 19638,
 8,
 5,
 2610,
 25336

In [5]:
SQUAD_MINDRECORD_FILE = "./data/train_features.mindrecord"

In [6]:
pad = lambda a,i : a[0:i] if len(a) > i else a + [0] * (i-len(a))
pad1 = lambda a,i : a[0:i] if len(a) > i else a + [1] * (i-len(a))
pad_entity = lambda a,i : a[0:i] if len(a) > i else np.append(a,[-1] * (i-len(a)))

for slist in list_dict:
    slist["entity_attention_mask"] = pad(slist["entity_attention_mask"], 24)
    slist["entity_ids"] = pad(slist["entity_attention_mask"], 24)
    slist["entity_segment_ids"] = pad(slist["entity_segment_ids"], 24)
    
    slist["word_ids"] = pad1(slist["word_ids"], 256)
    slist["word_segment_ids"] = pad(slist["word_segment_ids"], 256)
    slist["word_attention_mask"] = pad(slist["word_attention_mask"], 256)
    # entity padding 1
    entity_size = len(slist["entity_position_ids"])
    slist["entity_position_ids"] = np.array(slist["entity_position_ids"])
    temp = [[-1]*24 for i in range(24)]
    for i in range(24):
        if i < entity_size-1:
            temp[i]=(pad_entity(slist["entity_position_ids"][i], 24))


    slist["entity_position_ids"] =temp
    # entity_padding 2
    #slist["entity_position_ids"] = np.array(slist["entity_position_ids"]).flatten()
    #slist["entity_position_ids"] = pad_entity(slist["entity_position_ids"], 24)


if os.path.exists(SQUAD_MINDRECORD_FILE):
    os.remove(SQUAD_MINDRECORD_FILE)
    os.remove(SQUAD_MINDRECORD_FILE + ".db")

writer = FileWriter(file_name=SQUAD_MINDRECORD_FILE, shard_num=1)

data_schema = {
    #"unique_id": {"type": "int32", "shape": [-1]},
    "word_ids": {"type": "int32", "shape": [-1]},
    "word_segment_ids": {"type": "int32", "shape": [-1]},
    "word_attention_mask": {"type": "int32", "shape": [-1]},
    "entity_ids": {"type": "int32", "shape": [-1]},
    "entity_position_ids": {"type": "int32", "shape": [24,24]}, # 
    "entity_segment_ids": {"type": "int32", "shape": [-1]},
    "entity_attention_mask": {"type": "int32", "shape": [-1]},
    "start_positions": {"type": "int32", "shape": [-1]},
    "end_positions": {"type": "int32", "shape": [-1]}
}
writer.add_schema(data_schema, "it is a preprocessed squad dataset")

data = []
i = 0
for item in list_dict:
    i += 1
    sample = {
        #"unique_id": np.array(item["unique_id"], dtype=np.int32),
        "word_ids": np.array(item["word_ids"], dtype=np.int32),
        "word_segment_ids": np.array(item["word_segment_ids"], dtype=np.int32),
        "word_attention_mask": np.array(item["word_attention_mask"], dtype=np.int32),
        "entity_ids": np.array(item["entity_ids"], dtype=np.int32),
        "entity_position_ids": np.array(item["entity_position_ids"], dtype=np.int32),
        "entity_segment_ids": np.array(item["entity_segment_ids"], dtype=np.int32),
        "entity_attention_mask": np.array(item["entity_attention_mask"], dtype=np.int32),
        "start_positions": np.array(item["start_positions"], dtype=np.int32),
        "end_positions": np.array(item["end_positions"], dtype=np.int32),
    }

    data.append(sample)
    #print(sample)
    if i % 10 == 0:
        writer.write_raw_data(data)
        data = []
print(data[0])
if data:
    writer.write_raw_data(data)

writer.commit()

{'word_ids': array([    0,  1740,    99,   343,   473,  1586,  1071, 19252,   904,
         786,  8287,  4871,     7, 15359, 14785,   257,   116,     2,
           2,    20,  1049,   758,  3062,  2754, 15359, 14785,   257,
           8,  4634, 15377,    16,     5, 24311,  6455,  9965,  1016,
        4414,     6,  2034,    59,   411,  8130,    36,   401,  6301,
          36,   246,     4,   406, 11163, 35122,    31,     5,   343,
        2100,     4, 23701,  1070,    30,     5,  5280, 10333,  4305,
           9, 15377,    24,    34,    80, 20531,     6,    65,  1897,
           8,    65,   758,     4,   497,  1455,     6,    59,   820,
         758,  8537,  4686, 15377,     7,    97, 11633,    11,  1005,
           6,  1817,     8,     5,  2367,   953,     6,     7,  1947,
         215,    25, 12275,     6,  3534,     6,   229, 14024,  2186,
           6,  2920,     6, 15398,     6, 18959, 19193,     6,  8086,
        5870,     6, 12600,     6,  2884,   139,     6,   226,  7333,
       

MSRStatus.SUCCESS

In [7]:
output_features = np.array(list_dict)

In [8]:
columns_list = ['word_ids',  'word_segment_ids', 'word_attention_mask',
               'entity_ids', 'entity_position_ids', 'entity_segment_ids',
               'entity_attention_mask', 'start_positions', 'end_positions']

#SQUAD_MINDRECORD_FILE = "./data/dev_features.mindrecord"
data_set = ds.MindDataset(dataset_file=SQUAD_MINDRECORD_FILE, columns_list=columns_list)
count = 0
for item in data_set.create_dict_iterator():
    #print(item)
    count += 1
print("Got {} samples".format(count))

Got 87777 samples


In [9]:
data_set = data_set.batch(2)# , drop_remainder=True
data_sample = next(data_set.create_dict_iterator())
data_sample

{'word_ids': Tensor(shape=[2, 256], dtype=Int32, value=
 [[  0,  20, 343 ...   1,   1,   1],
  [  0, 653,  32 ...   1,   1,   1]]),
 'word_segment_ids': Tensor(shape=[2, 256], dtype=Int32, value=
 [[0, 0, 0 ... 0, 0, 0],
  [0, 0, 0 ... 0, 0, 0]]),
 'word_attention_mask': Tensor(shape=[2, 256], dtype=Int32, value=
 [[1, 1, 1 ... 0, 0, 0],
  [1, 1, 1 ... 0, 0, 0]]),
 'entity_ids': Tensor(shape=[2, 24], dtype=Int32, value=
 [[1, 1, 1 ... 0, 0, 0],
  [0, 0, 0 ... 0, 0, 0]]),
 'entity_position_ids': Tensor(shape=[2, 24, 24], dtype=Int32, value=
 [[[59, 60, 61 ... -1, -1, -1],
   [64, 65, 66 ... -1, -1, -1],
   [76, 77, -1 ... -1, -1, -1],
   ...
   [-1, -1, -1 ... -1, -1, -1],
   [-1, -1, -1 ... -1, -1, -1],
   [-1, -1, -1 ... -1, -1, -1]],
  [[-1, -1, -1 ... -1, -1, -1],
   [-1, -1, -1 ... -1, -1, -1],
   [-1, -1, -1 ... -1, -1, -1],
   ...
   [-1, -1, -1 ... -1, -1, -1],
   [-1, -1, -1 ... -1, -1, -1],
   [-1, -1, -1 ... -1, -1, -1]]]),
 'entity_segment_ids': Tensor(shape=[2, 24], dtype=I

In [10]:
data_sample['entity_position_ids']

Tensor(shape=[2, 24, 24], dtype=Int32, value=
[[[59, 60, 61 ... -1, -1, -1],
  [64, 65, 66 ... -1, -1, -1],
  [76, 77, -1 ... -1, -1, -1],
  ...
  [-1, -1, -1 ... -1, -1, -1],
  [-1, -1, -1 ... -1, -1, -1],
  [-1, -1, -1 ... -1, -1, -1]],
 [[-1, -1, -1 ... -1, -1, -1],
  [-1, -1, -1 ... -1, -1, -1],
  [-1, -1, -1 ... -1, -1, -1],
  ...
  [-1, -1, -1 ... -1, -1, -1],
  [-1, -1, -1 ... -1, -1, -1],
  [-1, -1, -1 ... -1, -1, -1]]])

In [11]:
data_sample['entity_attention_mask']

Tensor(shape=[2, 24], dtype=Int32, value=
[[1, 1, 1 ... 0, 0, 0],
 [0, 0, 0 ... 0, 0, 0]])

## Train

In [12]:
from readingcomprehension.models.luke import LukeForReadingComprehension, LukeForReadingComprehensionWithLoss, LukeEntityAwareAttentionModel, LukeSquadCell
import mindspore.common.dtype as mstype
from model.bert_model import BertConfig
from mindspore import context, save_checkpoint
from model.luke import LukeModel, EntityAwareEncoder
from mindspore import Tensor, context
from mindspore import dtype as mstype
import mindspore.ops as ops
import mindspore.nn as nn
from model.bert_model import BertOutput
from mindspore.common.initializer import TruncatedNormal
from mindspore.ops import composite as C
import mindspore
from mindspore.ops import operations as P
from mindspore.train.model import Model
from tqdm import tqdm
from mindspore.train.serialization import load_checkpoint, load_param_into_net
from mindspore.train.model import Model
import collections
# PYNATIVE_MODE & GRAPH_MODE
context.set_context(mode=context.GRAPH_MODE, device_target="GPU")
context.set_context(enable_graph_kernel=True)

In [13]:

# def do_train(dataset=None, network=None, load_checkpoint_path="", save_checkpoint_path="", epoch_num=1):
#     """ do train """
#     if load_checkpoint_path == "":
#         raise ValueError("Pretrain model missed, finetune task must load pretrain model!")
#     steps_per_epoch = dataset.get_dataset_size()
#     # optimizer
#     if optimizer_cfg.optimizer == 'AdamWeightDecay':
#         lr_schedule = BertLearningRate(learning_rate=15e-6,
#                                        end_learning_rate=optimizer_cfg.AdamWeightDecay.end_learning_rate,
#                                        warmup_steps=int(steps_per_epoch * epoch_num * 0.1),
#                                        decay_steps=steps_per_epoch * epoch_num,
#                                        power=optimizer_cfg.AdamWeightDecay.power)
#         params = network.trainable_params()
#         decay_params = list(filter(optimizer_cfg.AdamWeightDecay.decay_filter, params))
#         other_params = list(filter(lambda x: not optimizer_cfg.AdamWeightDecay.decay_filter(x), params))
#         group_params = [{'params': decay_params, 'weight_decay': optimizer_cfg.AdamWeightDecay.weight_decay},
#                         {'params': other_params, 'weight_decay': 0.0}]

#         optimizer = AdamWeightDecay(group_params, lr_schedule, eps=optimizer_cfg.AdamWeightDecay.eps)
#     elif optimizer_cfg.optimizer == 'Lamb':
#         lr_schedule = BertLearningRate(learning_rate=optimizer_cfg.Lamb.learning_rate,
#                                        end_learning_rate=optimizer_cfg.Lamb.end_learning_rate,
#                                        warmup_steps=int(steps_per_epoch * epoch_num * 0.1),
#                                        decay_steps=steps_per_epoch * epoch_num,
#                                        power=optimizer_cfg.Lamb.power)
#         optimizer = Lamb(network.trainable_params(), learning_rate=lr_schedule)
#     elif optimizer_cfg.optimizer == 'Momentum':
#         optimizer = Momentum(network.trainable_params(), learning_rate=optimizer_cfg.Momentum.learning_rate,
#                              momentum=optimizer_cfg.Momentum.momentum)
#     else:
#         raise Exception("Optimizer not supported. support: [AdamWeightDecay, Lamb, Momentum]")

#     # load checkpoint into network
#     ckpt_config = CheckpointConfig(save_checkpoint_steps=steps_per_epoch, keep_checkpoint_max=1)
#     ckpoint_cb = ModelCheckpoint(prefix="squad",
#                                  directory=None if save_checkpoint_path == "" else save_checkpoint_path,
#                                  config=ckpt_config)
#     param_dict = load_checkpoint(load_checkpoint_path)
#     load_param_into_net(network, param_dict)

#     update_cell = DynamicLossScaleUpdateCell(loss_scale_value=2 ** 32, scale_factor=2, scale_window=1000)
#     netwithgrads = BertSquadCell(network, optimizer=optimizer, scale_update_cell=update_cell)
#     model = Model(netwithgrads)
#     callbacks = [TimeMonitor(dataset.get_dataset_size()), LossCallBack(dataset.get_dataset_size()), ckpoint_cb]
#     model.train(epoch_num, dataset, callbacks=callbacks)

In [14]:
from mindspore.train.callback import Callback
from mindspore.train.serialization import load_checkpoint, load_param_into_net
from mindspore.train.model import Model
from mindspore.nn.learning_rate_schedule import LearningRateSchedule, PolynomialDecayLR, WarmUpLR
from mindspore.nn.wrap.loss_scale import DynamicLossScaleUpdateCell

import collections
import time
class BertLearningRate(LearningRateSchedule):
    """
    Warmup-decay learning rate for Bert network.
    """

    def __init__(self, learning_rate, end_learning_rate, warmup_steps, decay_steps, power):
        super(BertLearningRate, self).__init__()
        self.warmup_flag = False
        if warmup_steps > 0:
            self.warmup_flag = True
            self.warmup_lr = WarmUpLR(learning_rate, warmup_steps)
        self.decay_lr = PolynomialDecayLR(learning_rate, end_learning_rate, decay_steps, power)
        self.warmup_steps = Tensor(np.array([warmup_steps]).astype(np.float32))

        self.greater = mindspore.ops.Greater()
        self.one = Tensor(np.array([1.0]).astype(np.float32))
        self.cast = mindspore.ops.Cast()

    def construct(self, global_step):
        decay_lr = self.decay_lr(global_step)
        if self.warmup_flag:
            is_warmup = self.cast(self.greater(self.warmup_steps, global_step), mstype.float32)
            warmup_lr = self.warmup_lr(global_step)
            lr = (self.one - is_warmup) * decay_lr + is_warmup * warmup_lr
        else:
            lr = decay_lr
        return lr


class LossCallBack(Callback):
    """
    Monitor the loss in training.

    If the loss is NAN or INF terminating training.

    Note:
        If per_print_times is 0 do not print loss.

    Args:
        per_print_times (int): Print loss every times. Default: 1.
    """

    def __init__(self, per_print_times=1, rank_ids=0):
        super(LossCallBack, self).__init__()
        if not isinstance(per_print_times, int) or per_print_times < 0:
            raise ValueError("print_step must be int and >= 0.")
        self._per_print_times = per_print_times
        self.rank_id = rank_ids
        self.time_stamp_first = get_ms_timestamp()

    def step_end(self, run_context):
        """Monitor the loss in training."""
        time_stamp_current = get_ms_timestamp()
        cb_params = run_context.original_args()
        print("time: {}, epoch: {}, step: {}, outputs are {}".format(time_stamp_current - self.time_stamp_first,
                                                                     cb_params.cur_epoch_num,
                                                                     cb_params.cur_step_num,
                                                                     str(cb_params.net_outputs)))
        with open("./loss_{}.log".format(self.rank_id), "a+") as f:
            f.write("time: {}, epoch: {}, step: {}, loss: {}".format(
                time_stamp_current - self.time_stamp_first,
                cb_params.cur_epoch_num,
                cb_params.cur_step_num,
                str(cb_params.net_outputs[0].asnumpy())))
            f.write('\n')


def get_ms_timestamp():
    t = time.time()
    return int(round(t * 1000))

In [15]:
for i in data_set:
    print(len(i))
    print(i)
    break

9
[Tensor(shape=[2, 256], dtype=Int32, value=
[[    0, 14000, 24280 ...     1,     1,     1],
 [    0,   520,    21 ...     1,     1,     1]]), Tensor(shape=[2, 256], dtype=Int32, value=
[[0, 0, 0 ... 0, 0, 0],
 [0, 0, 0 ... 0, 0, 0]]), Tensor(shape=[2, 256], dtype=Int32, value=
[[1, 1, 1 ... 0, 0, 0],
 [1, 1, 1 ... 0, 0, 0]]), Tensor(shape=[2, 24], dtype=Int32, value=
[[1, 0, 0 ... 0, 0, 0],
 [0, 0, 0 ... 0, 0, 0]]), Tensor(shape=[2, 24, 24], dtype=Int32, value=
[[[72, 73, -1 ... -1, -1, -1],
  [-1, -1, -1 ... -1, -1, -1],
  [-1, -1, -1 ... -1, -1, -1],
  ...
  [-1, -1, -1 ... -1, -1, -1],
  [-1, -1, -1 ... -1, -1, -1],
  [-1, -1, -1 ... -1, -1, -1]],
 [[-1, -1, -1 ... -1, -1, -1],
  [-1, -1, -1 ... -1, -1, -1],
  [-1, -1, -1 ... -1, -1, -1],
  ...
  [-1, -1, -1 ... -1, -1, -1],
  [-1, -1, -1 ... -1, -1, -1],
  [-1, -1, -1 ... -1, -1, -1]]]), Tensor(shape=[2, 24], dtype=Int32, value=
[[0, 0, 0 ... 0, 0, 0],
 [0, 0, 0 ... 0, 0, 0]]), Tensor(shape=[2, 24], dtype=Int32, value=
[[1, 0, 0 

In [ ]:
epoch=1

luke_config = BertConfig()
LUKEModel = LukeForReadingComprehension(luke_config)
param_dict = load_checkpoint('./luke-large-qa.ckpt')
load_param_into_net(LUKEModel,param_dict)

# update_cell = DynamicLossScaleUpdateCell(loss_scale_value=2 ** 32, scale_factor=2, scale_window=1000)

lukesquad = LukeForReadingComprehensionWithLoss(LUKEModel)

# lr_schedule = BertLearningRate(learning_rate=15e-6,
#                                    end_learning_rate=15e-6 * 0,
#                                    warmup_steps=int(data_set.get_dataset_size() * epoch * 0.1),
#                                    decay_steps=data_set.get_dataset_size() * epoch,
#                                    power=1.0)

update_cell = DynamicLossScaleUpdateCell(loss_scale_value=2 ** 32, scale_factor=2, scale_window=1000)


params = lukesquad.trainable_params()

optimizer = mindspore.nn.AdamWeightDecay(params,
                                         learning_rate=5e-5, #15e-6
                                         beta1=0.9,
                                         beta2=0.98,
                                         eps=1e-06)

# optimizer = mindspore.nn.Adam(params, learning_rate=lr_schedule, eps=1e-8)

# lr_schedule
# warmup_steps=877
num_train_steps=14629
warmup_steps = int(epoch * num_train_steps * 0.06)
netwithgrads = LukeSquadCell(lukesquad,optimizer=optimizer, scale_update_cell=update_cell)
model = Model(netwithgrads)

loss_monitor = LossCallBack()
model.train(epoch,data_set,callbacks=[loss_monitor],dataset_sink_mode=False)




time: 76862, epoch: 1, step: 1, outputs are (Tensor(shape=[], dtype=Float32, value= 0.589003), Tensor(shape=[], dtype=Bool, value= False))
time: 77060, epoch: 1, step: 2, outputs are (Tensor(shape=[], dtype=Float32, value= 1.21951), Tensor(shape=[], dtype=Bool, value= False))
time: 77264, epoch: 1, step: 3, outputs are (Tensor(shape=[], dtype=Float32, value= 0.009493), Tensor(shape=[], dtype=Bool, value= False))
time: 77462, epoch: 1, step: 4, outputs are (Tensor(shape=[], dtype=Float32, value= 0.363278), Tensor(shape=[], dtype=Bool, value= False))
time: 77664, epoch: 1, step: 5, outputs are (Tensor(shape=[], dtype=Float32, value= 0.113523), Tensor(shape=[], dtype=Bool, value= False))
time: 77861, epoch: 1, step: 6, outputs are (Tensor(shape=[], dtype=Float32, value= 3.7394), Tensor(shape=[], dtype=Bool, value= False))
time: 78061, epoch: 1, step: 7, outputs are (Tensor(shape=[], dtype=Float32, value= 0.626775), Tensor(shape=[], dtype=Bool, value= False))
time: 78264, epoch: 1, step: 8

time: 88616, epoch: 1, step: 60, outputs are (Tensor(shape=[], dtype=Float32, value= 2.6534), Tensor(shape=[], dtype=Bool, value= False))
time: 88813, epoch: 1, step: 61, outputs are (Tensor(shape=[], dtype=Float32, value= 5.19086), Tensor(shape=[], dtype=Bool, value= False))
time: 89012, epoch: 1, step: 62, outputs are (Tensor(shape=[], dtype=Float32, value= 0.70108), Tensor(shape=[], dtype=Bool, value= False))
time: 89212, epoch: 1, step: 63, outputs are (Tensor(shape=[], dtype=Float32, value= 1.53523), Tensor(shape=[], dtype=Bool, value= False))
time: 89411, epoch: 1, step: 64, outputs are (Tensor(shape=[], dtype=Float32, value= -0.961953), Tensor(shape=[], dtype=Bool, value= False))
time: 89608, epoch: 1, step: 65, outputs are (Tensor(shape=[], dtype=Float32, value= 1.42607), Tensor(shape=[], dtype=Bool, value= False))
time: 89805, epoch: 1, step: 66, outputs are (Tensor(shape=[], dtype=Float32, value= 1.46644), Tensor(shape=[], dtype=Bool, value= False))
time: 90003, epoch: 1, ste

time: 100559, epoch: 1, step: 119, outputs are (Tensor(shape=[], dtype=Float32, value= 3.03344), Tensor(shape=[], dtype=Bool, value= False))
time: 100773, epoch: 1, step: 120, outputs are (Tensor(shape=[], dtype=Float32, value= 2.8982), Tensor(shape=[], dtype=Bool, value= False))
time: 100972, epoch: 1, step: 121, outputs are (Tensor(shape=[], dtype=Float32, value= 2.3709), Tensor(shape=[], dtype=Bool, value= False))
time: 101172, epoch: 1, step: 122, outputs are (Tensor(shape=[], dtype=Float32, value= 0.621152), Tensor(shape=[], dtype=Bool, value= False))
time: 101370, epoch: 1, step: 123, outputs are (Tensor(shape=[], dtype=Float32, value= 3.13385), Tensor(shape=[], dtype=Bool, value= False))
time: 101571, epoch: 1, step: 124, outputs are (Tensor(shape=[], dtype=Float32, value= 1.93239), Tensor(shape=[], dtype=Bool, value= False))
time: 101784, epoch: 1, step: 125, outputs are (Tensor(shape=[], dtype=Float32, value= 1.99541), Tensor(shape=[], dtype=Bool, value= False))
time: 101998, 

time: 112452, epoch: 1, step: 178, outputs are (Tensor(shape=[], dtype=Float32, value= 0.729395), Tensor(shape=[], dtype=Bool, value= False))
time: 112664, epoch: 1, step: 179, outputs are (Tensor(shape=[], dtype=Float32, value= 0.592571), Tensor(shape=[], dtype=Bool, value= False))
time: 112873, epoch: 1, step: 180, outputs are (Tensor(shape=[], dtype=Float32, value= 1.81153), Tensor(shape=[], dtype=Bool, value= False))
time: 113076, epoch: 1, step: 181, outputs are (Tensor(shape=[], dtype=Float32, value= 1.40491), Tensor(shape=[], dtype=Bool, value= False))
time: 113279, epoch: 1, step: 182, outputs are (Tensor(shape=[], dtype=Float32, value= 1.27681), Tensor(shape=[], dtype=Bool, value= False))
time: 113481, epoch: 1, step: 183, outputs are (Tensor(shape=[], dtype=Float32, value= 2.76085), Tensor(shape=[], dtype=Bool, value= False))
time: 113685, epoch: 1, step: 184, outputs are (Tensor(shape=[], dtype=Float32, value= 0.0371542), Tensor(shape=[], dtype=Bool, value= False))
time: 113

time: 124341, epoch: 1, step: 237, outputs are (Tensor(shape=[], dtype=Float32, value= 0.417153), Tensor(shape=[], dtype=Bool, value= False))
time: 124540, epoch: 1, step: 238, outputs are (Tensor(shape=[], dtype=Float32, value= 4.65154), Tensor(shape=[], dtype=Bool, value= False))
time: 124740, epoch: 1, step: 239, outputs are (Tensor(shape=[], dtype=Float32, value= 0.54357), Tensor(shape=[], dtype=Bool, value= False))
time: 124940, epoch: 1, step: 240, outputs are (Tensor(shape=[], dtype=Float32, value= 1.26902), Tensor(shape=[], dtype=Bool, value= False))
time: 125140, epoch: 1, step: 241, outputs are (Tensor(shape=[], dtype=Float32, value= 4.60995), Tensor(shape=[], dtype=Bool, value= False))
time: 125340, epoch: 1, step: 242, outputs are (Tensor(shape=[], dtype=Float32, value= 0.220308), Tensor(shape=[], dtype=Bool, value= False))
time: 125542, epoch: 1, step: 243, outputs are (Tensor(shape=[], dtype=Float32, value= 1.74382), Tensor(shape=[], dtype=Bool, value= False))
time: 12574

...

time: 337597, epoch: 1, step: 1291, outputs are (Tensor(shape=[], dtype=Float32, value= 1.96099), Tensor(shape=[], dtype=Bool, value= False))
time: 337800, epoch: 1, step: 1292, outputs are (Tensor(shape=[], dtype=Float32, value= 4.05078), Tensor(shape=[], dtype=Bool, value= False))
time: 338004, epoch: 1, step: 1293, outputs are (Tensor(shape=[], dtype=Float32, value= 1.67928), Tensor(shape=[], dtype=Bool, value= False))
time: 338205, epoch: 1, step: 1294, outputs are (Tensor(shape=[], dtype=Float32, value= 2.00415), Tensor(shape=[], dtype=Bool, value= False))
time: 338406, epoch: 1, step: 1295, outputs are (Tensor(shape=[], dtype=Float32, value= 0.793025), Tensor(shape=[], dtype=Bool, value= False))
time: 338610, epoch: 1, step: 1296, outputs are (Tensor(shape=[], dtype=Float32, value= 0.25753), Tensor(shape=[], dtype=Bool, value= False))
time: 338812, epoch: 1, step: 1297, outputs are (Tensor(shape=[], dtype=Float32, value= 3.21598), Tensor(shape=[], dtype=Bool, value= False))
time:

time: 349540, epoch: 1, step: 1350, outputs are (Tensor(shape=[], dtype=Float32, value= 0.844658), Tensor(shape=[], dtype=Bool, value= False))
time: 349741, epoch: 1, step: 1351, outputs are (Tensor(shape=[], dtype=Float32, value= 3.0454), Tensor(shape=[], dtype=Bool, value= False))
time: 349943, epoch: 1, step: 1352, outputs are (Tensor(shape=[], dtype=Float32, value= 0.884346), Tensor(shape=[], dtype=Bool, value= False))
time: 350144, epoch: 1, step: 1353, outputs are (Tensor(shape=[], dtype=Float32, value= 2.15012), Tensor(shape=[], dtype=Bool, value= False))
time: 350345, epoch: 1, step: 1354, outputs are (Tensor(shape=[], dtype=Float32, value= 2.29434), Tensor(shape=[], dtype=Bool, value= False))
time: 350547, epoch: 1, step: 1355, outputs are (Tensor(shape=[], dtype=Float32, value= 1.38006), Tensor(shape=[], dtype=Bool, value= False))
time: 350753, epoch: 1, step: 1356, outputs are (Tensor(shape=[], dtype=Float32, value= 1.41261), Tensor(shape=[], dtype=Bool, value= False))
time:

time: 361391, epoch: 1, step: 1408, outputs are (Tensor(shape=[], dtype=Float32, value= 2.8374), Tensor(shape=[], dtype=Bool, value= False))
time: 361600, epoch: 1, step: 1409, outputs are (Tensor(shape=[], dtype=Float32, value= 3.7596), Tensor(shape=[], dtype=Bool, value= False))
time: 361805, epoch: 1, step: 1410, outputs are (Tensor(shape=[], dtype=Float32, value= 3.85918), Tensor(shape=[], dtype=Bool, value= False))
time: 362008, epoch: 1, step: 1411, outputs are (Tensor(shape=[], dtype=Float32, value= 3.90175), Tensor(shape=[], dtype=Bool, value= False))
time: 362210, epoch: 1, step: 1412, outputs are (Tensor(shape=[], dtype=Float32, value= 2.2214), Tensor(shape=[], dtype=Bool, value= False))
time: 362420, epoch: 1, step: 1413, outputs are (Tensor(shape=[], dtype=Float32, value= 3.72805), Tensor(shape=[], dtype=Bool, value= False))
time: 362629, epoch: 1, step: 1414, outputs are (Tensor(shape=[], dtype=Float32, value= 0.535683), Tensor(shape=[], dtype=Bool, value= False))
time: 36

time: 373567, epoch: 1, step: 1466, outputs are (Tensor(shape=[], dtype=Float32, value= 0.940696), Tensor(shape=[], dtype=Bool, value= False))
time: 373772, epoch: 1, step: 1467, outputs are (Tensor(shape=[], dtype=Float32, value= 2.04834), Tensor(shape=[], dtype=Bool, value= False))
time: 373975, epoch: 1, step: 1468, outputs are (Tensor(shape=[], dtype=Float32, value= 1.64436), Tensor(shape=[], dtype=Bool, value= False))
time: 374181, epoch: 1, step: 1469, outputs are (Tensor(shape=[], dtype=Float32, value= 0.461405), Tensor(shape=[], dtype=Bool, value= False))
time: 374392, epoch: 1, step: 1470, outputs are (Tensor(shape=[], dtype=Float32, value= 0.296272), Tensor(shape=[], dtype=Bool, value= False))
time: 374595, epoch: 1, step: 1471, outputs are (Tensor(shape=[], dtype=Float32, value= 0.328559), Tensor(shape=[], dtype=Bool, value= False))
time: 374801, epoch: 1, step: 1472, outputs are (Tensor(shape=[], dtype=Float32, value= 1.26547), Tensor(shape=[], dtype=Bool, value= False))
ti

time: 385474, epoch: 1, step: 1524, outputs are (Tensor(shape=[], dtype=Float32, value= 0.756883), Tensor(shape=[], dtype=Bool, value= False))
time: 385676, epoch: 1, step: 1525, outputs are (Tensor(shape=[], dtype=Float32, value= 2.81864), Tensor(shape=[], dtype=Bool, value= False))
time: 385879, epoch: 1, step: 1526, outputs are (Tensor(shape=[], dtype=Float32, value= 1.34027), Tensor(shape=[], dtype=Bool, value= False))
time: 386083, epoch: 1, step: 1527, outputs are (Tensor(shape=[], dtype=Float32, value= 2.05863), Tensor(shape=[], dtype=Bool, value= False))
time: 386285, epoch: 1, step: 1528, outputs are (Tensor(shape=[], dtype=Float32, value= 0.709875), Tensor(shape=[], dtype=Bool, value= False))
time: 386488, epoch: 1, step: 1529, outputs are (Tensor(shape=[], dtype=Float32, value= 0.310592), Tensor(shape=[], dtype=Bool, value= False))
time: 386691, epoch: 1, step: 1530, outputs are (Tensor(shape=[], dtype=Float32, value= 0.616093), Tensor(shape=[], dtype=Bool, value= False))
ti

In [ ]:

save_checkpoint(model.train_network.network.net, ckpt_path / Path("ft.ckpt"))